# Human Alignment Project Demo

This notebook demonstrates how to use the `HumanAlignment` class to:
1. Initialize and prepare resources.
2. Compile and run the AutoSxS pipeline.
3. Fetch evaluation results and clean up resources.

In [1]:
!pip install google-cloud-aiplatform google-cloud-pipeline-components pandas kfp

## Step 1: Initialize HumanAlignment Class

In [2]:
from human_alignment import HumanAlignment

# Define project settings
project_id = "your-project-id"
location = "us-central1"
bucket_name = None  # Optional: Leave as None to auto-generate a bucket name

# Initialize the class
alignment = HumanAlignment(project_id=project_id, location=location, bucket_name=bucket_name)

## Step 2: Create Cloud Storage Bucket

In [3]:
# Create the bucket
alignment.create_bucket()

## Step 3: Prepare Evaluation Dataset

In [4]:
# Define dataset inputs
context = [
    "Context for example 1",
    "Context for example 2",
]
questions = [
    "What is the first question?",
    "What is the second question?",
]
predictions_a = [
    "Prediction from model A for example 1",
    "Prediction from model A for example 2",
]
predictions_b = [
    "Prediction from model B for example 1",
    "Prediction from model B for example 2",
]
human_preference = ["A", "B"]  # Human preferences for each example

# Upload the dataset
alignment.prepare_dataset(context, questions, predictions_a, predictions_b, human_preference)

## Step 4: Compile the Pipeline

In [5]:
from google_cloud_pipeline_components.v1 import model_evaluation

# Compile the pipeline
alignment.compile_pipeline(pipeline_func=model_evaluation.autosxs_pipeline)

## Step 5: Run the Pipeline

In [6]:
# Define pipeline parameters
parameters = {
    "evaluation_dataset": alignment.dataset_path,
    "id_columns": ["questions"],
    "autorater_prompt_parameters": {
        "inference_context": {"column": "context"},
        "inference_instruction": {"column": "questions"},
    },
    "task": "question_answering",
    "response_column_a": "pred_a",
    "response_column_b": "pred_b",
    "human_preference_column": "actuals",
}

# Run the pipeline
display_name = "human-alignment-demo"
job = alignment.run_pipeline(parameters, display_name=display_name)

## Step 6: Fetch Results

In [7]:
# Fetch the results
outputs = alignment.fetch_job_outputs(job, task_name="online-evaluation-pairwise")
judgments_uri = outputs["judgments"].artifacts[0].uri

# Load and display judgments
judgments_df = alignment.load_metrics(judgments_uri)
judgments_df.head()

## Step 7: Clean Up Resources

In [8]:
# Clean up resources (Optional)
alignment.clean_up(delete_bucket=True)